# Notebook Instructions

1. All the <u>code and data files</u> used in this course are available in the downloadable unit of the <u>last section of this course</u>.
2. You can run the notebook document sequentially (one cell at a time) by pressing **shift + enter**. 
3. While a cell is running, a [*] is shown on the left. After the cell is run, the output will appear on the next line.

This course is based on specific versions of python packages. You can find the details of the packages in <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank" >this manual</a>.

# Fama-French Three-Factor Model

In this notebook, you will learn to calculate the expected returns of the stock using Fama-French three-factor Model.

The steps for the same:

1. Read the data from CSV
2. Calculate the daily and excess returns of stock
3. Calculate the coefficients of Fama-French factors
4. Calculate the annualised returns of Fama-French factors
5. Calculate the expected returns of stock 

![title](https://d2a032ejo53cab.cloudfront.net/Glossary/qbbVNWuq/FF.PNG)

### Read the data from CSV
The `read_csv()` function is used to read the data from the CSV file. We will read the data for excess market returns (Rm-Rf), Small minus Big (SMB), High minus Low (HML), risk-free rate (Rf) and the price of the Amazon (AMZN). The data we are using for the calculation of expected returns is from 02-June-2016 to 30-April-2019. This is arbitrarily chosen period for illustration purpose.

We have downloaded the data for excess market returns (Rm-Rf), Small minus Big (SMB), High minus Low (HML), and risk-free rate (Rf) from [here](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).

In [1]:
# Import the pandas library
import pandas as pd

# Read the data
data = pd.read_csv('../data_modules/Data_2016_to_2019_FF.csv',
                   index_col=0, parse_dates=['date'])

# Print first five rows of data
data.tail()

,market_excess_returns,SMB,HML,risk_free_rate,amazon_price
date,,,,,
2019-04-24,-0.23,0.31,-0.22,0.01,207.16
2019-04-25,-0.14,-0.70,-0.27,0.01,205.28
2019-04-26,0.53,0.43,0.05,0.01,204.30
2019-04-29,0.18,0.12,0.61,0.01,204.61
2019-04-30,-0.04,-0.71,0.27,0.01,200.67


### Calculate the daily Amazon returns

The `pct_change()` function is used to calculate the daily percentage change

In [2]:
# Calculate the daily returns of Amazon
data['daily_amazon_returns'] = data.amazon_price.pct_change() * 100

# Drop the NaN values
data = data.dropna()

# Print first five rows of data
data.head()

,market_excess_returns,SMB,HML,risk_free_rate,amazon_price,daily_amazon_returns
date,,,,,,
2016-05-03,-1.05,-0.66,-0.49,0.001,95.18,1.644596
2016-05-04,-0.66,-0.15,0.06,0.001,94.19,-1.040134
2016-05-05,-0.08,-0.50,0.09,0.001,93.24,-1.008600
2016-05-06,0.38,0.05,0.24,0.001,92.72,-0.557701
2016-05-09,0.05,0.36,-1.48,0.001,92.79,0.075496


### Calculate the Amazon excess returns
Amazon excess returns are calculated by subtracting risk-free rate from daily Amazon returns. 


In [3]:
# Calculate the Amazon excess returns
data['amazon_excess_returns'] = data['daily_amazon_returns'] - \
    data['risk_free_rate']

# Print first five rows of data
data.head()

,market_excess_returns,SMB,HML,risk_free_rate,amazon_price,daily_amazon_returns,amazon_excess_returns
date,,,,,,,
2016-05-03,-1.05,-0.66,-0.49,0.001,95.18,1.644596,1.643596
2016-05-04,-0.66,-0.15,0.06,0.001,94.19,-1.040134,-1.041134
2016-05-05,-0.08,-0.50,0.09,0.001,93.24,-1.008600,-1.009600
2016-05-06,0.38,0.05,0.24,0.001,92.72,-0.557701,-0.558701
2016-05-09,0.05,0.36,-1.48,0.001,92.79,0.075496,0.074496


### Calculate the coefficients using regression method
We will use the ordinary least square (OLS) regression method to calculate the coefficients or regression beta of market_excess_returns, SMB, and HML. 

For that, we will use the OLS method from statsmodels library.

Syntax:

`OLS(y, X)`

In our case, `y` = amazon_excess_returns and `X` = market_excess_returns, SMB, and HML

In [4]:
# Import the statsmodels library
import statsmodels.api as sm

# Independent variables
X = data[['market_excess_returns', 'SMB', 'HML']]

# Dependent variable
y = data['amazon_excess_returns']

# Create a regression model
reg = sm.OLS(y, X).fit()

# Print beta of market_excess_returns
print('Beta of market_excess_returns= ', reg.params[0])

# Print beta of SMB
print('Beta of SMB= ', reg.params[1])

# Print beta of HML
print('Beta of HML= ', reg.params[2])

Beta of market_excess_returns=  1.154504672981659
Beta of SMB=  -0.27358778926285215
Beta of HML=  -0.7171104348708256


### Calculate the annualised return of market excess returns, SMB returns, HML returns, risk-free rate returns

In [5]:
# Calculate the annualised market excess returns
annualised_market_excess_returns = (
    (data.market_excess_returns/100 + 1).cumprod()[-1]) ** (252/len(data)) - 1

# Print annualised market excess returns
annualised_market_excess_returns

0.13889333362659628

In [6]:
# Calculate the annualised SMB returns
annualised_SMB_returns = ((data.SMB/100 + 1).cumprod()
                          [-1]) ** (252/len(data)) - 1

# Print annualised_SMB_returns
annualised_SMB_returns

-0.002627390164124943

In [7]:
# Calculate the annualised HML returns
annualised_HML_returns = ((data.HML/100 + 1).cumprod()
                          [-1]) ** (252/len(data)) - 1

# Print annualised_HML_returns
annualised_HML_returns

-0.048414947762249416

In [8]:
# Calculate annualised risk-free rate returns
annualised_risk_free_rate_returns = (
    data.risk_free_rate/100 + 1).cumprod()[-1] ** (252/len(data)) - 1

# Print annualised_risk_free_rate_returns
annualised_risk_free_rate_returns

0.01196786051367904

### Calculate the expected returns of Amazon

Expected returns of Amazon (Ri) = Risk-free rate (Rf) + βm * market excess returns + βSMB * SMB + βHML * HML

In [9]:
# Calculate the expected returns of Amazon
expected_returns_amazon = annualised_risk_free_rate_returns + \
    reg.params[0] * annualised_market_excess_returns + reg.params[1] * \
    annualised_SMB_returns + reg.params[2] * annualised_HML_returns

# Print expected returns of Amazon
print('Expected Returns of Amazon: %.2f' % (expected_returns_amazon * 100))

Expected Returns of Amazon: 20.78


As you can see the expected returns of Amazon for 1 year using the Fama-French three-factor model comes out to be 20.78%. In the upcoming units, you will practice these codes through interactive exercises.<br><br>